In [ ]:
import socket
import time
import threading
from pop import Gas, Dust, TempHumi, Pir, PiezoBuzzer, Touch, Led, PwmController, Textlcd

# ==========================================
# 1. 설정 및 초기화
# ==========================================
JAVA_IP = "192.168.101.102"
PORT_CMD    = 39186
PORT_SENSOR = 39187
PORT_DOOR   = 39189

# 객체 초기화
gas = Gas(6)
dust = Dust()
th = TempHumi()
pir = Pir(22)

PIN_BUZZER = 12
PIN_STATUS_LED = 27
lcd = Textlcd()
pwm = PwmController()
pwm.init()
pwm.setChannel(4)
touch = Touch()
p = PiezoBuzzer(PIN_BUZZER)
leds = Led(PIN_STATUS_LED)

# 변수
password = [10, 2, 10, 4]
input_pw = []
fail_count = 0
is_locked = True

# ==========================================
# [핵심 수정] 안전한 비프음 함수
# ==========================================
def beep_touch():
    """터치했을 때 짧게 삑! 소리 내기"""
    try:
        p.tone(4, 4, 10)  # 소리 켜기
        time.sleep(0.1)   # 0.1초 대기
        p.piezo.ChangeDutyCycle(0) # stop() 대신 듀티를 0으로 (음소거)
    except:
        pass

def play_tone(high):
    """성공/실패 알림음"""
    try:
        p.piezo.ChangeDutyCycle(0) # 일단 음소거
        
        if high:
            # 성공음 (도 -> 미 -> 솔 느낌)
            p.tone(5, 1, 10) # 도
            time.sleep(0.1)
            p.tone(5, 5, 10) # 미
            time.sleep(0.1)
            p.tone(5, 8, 10) # 솔
        else:
            # 실패음 (삐- 삐-)
            p.tone(3, 1, 10)
            time.sleep(0.2)
            p.piezo.ChangeDutyCycle(0)
            time.sleep(0.1)
            p.tone(3, 1, 10)
            
        time.sleep(0.3)
        p.piezo.ChangeDutyCycle(0) # stop() 대신 음소거
    except:
        pass

# ==========================================
# 2. 도어락 제어 및 통신
# ==========================================
def send_door_event(event):
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(0.5)
        s.connect((JAVA_IP, PORT_DOOR))
        s.sendall((event + "\n").encode())
        s.close()
    except: pass

def lock_door():
    global is_locked
    print("\n[ACTION] 문 잠김")
    is_locked = True
    send_door_event("LOCKED")
    try:
        leds.off()
        pwm.setFreq(50)
        pwm.setDuty((1.6 / 20) * 100)
        lcd.clear()
        lcd.print("LOCKED")
    except: pass

def unlock_door():
    global is_locked, fail_count
    print("\n[ACTION] 문 열림! (8초 후 자동 잠금)")
    is_locked = False
    fail_count = 0
    send_door_event("UNLOCKED")
    try:
        leds.on()
        pwm.setFreq(50)
        pwm.setDuty((0.6 / 20) * 100)
        lcd.clear()
        lcd.print("OPEN")
    except: pass
    
    threading.Timer(8.0, lock_door).start()

# ==========================================
# 3. 서버 리스너 (명령 수신)
# ==========================================
def command_listener_job():
    print(f"[Thread] 명령 서버 대기중...")
    while True:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.connect((JAVA_IP, PORT_CMD))
            print("[System] Java 서버 연결됨")
            while True:
                raw_data = sock.recv(1024).decode()
                if not raw_data: break
                for cmd in raw_data.split('\n'):
                    cmd = cmd.strip().upper()
                    if not cmd: continue
                    print(f"[명령 수신] {cmd}")
                    
                    if cmd == "UNLOCK": unlock_door()
                    elif cmd == "LOCK": lock_door()
                    elif cmd == "LED_ON": leds.on()
                    elif cmd == "LED_OFF": leds.off()
                    elif cmd == "REQ_FACE_UNLOCK": pass
            sock.close()
        except: time.sleep(2)

t_cmd = threading.Thread(target=command_listener_job, daemon=True)
t_cmd.start()

# ==========================================
# 4. 센서 데이터 전송
# ==========================================
def sensor_thread_job():
    while True:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.connect((JAVA_IP, PORT_SENSOR))
            break
        except: time.sleep(2)

    while True:
        try:
            try:
                g_val = gas.readAverage()
                methan = gas.calcMethan(g_val)
                dust.read()
                pm1, pm25, pm10 = dust.pm_1p0_grimm, dust.pm_2p5_grimm, dust.pm_10_grimm
                temp = th.getTemperature()
                humi = th.getHumidity()
                pir_val = pir.read()
            except: pm1, pm25, pm10, temp, humi, pir_val, g_val, methan = 0,0,0,0,0,0,0,0

            msg = f"SENSOR GAS={g_val} METHAN={methan} TEMP={temp:.2f} HUMI={humi:.2f} PM1={pm1} PM25={pm25} PM10={pm10} PIR={pir_val}\n"
            sock.sendall(msg.encode())
            time.sleep(1)
        except: break

t_sensor = threading.Thread(target=sensor_thread_job, daemon=True)
t_sensor.start()

# ==========================================
# 5. 메인 루프 (키패드)
# ==========================================
print("\n[Main] 시스템 가동... (터치 대기)")
lock_door()

try:
    while True:
        for ch in range(12):
            if touch.readChannel(ch) == 1:
                key_char = '*' if ch == 9 else ('#' if ch == 11 else str(ch))
                print(f"키 입력: {key_char}")

                # 터치음 함수 호출
                beep_touch()

                input_pw.append(ch)

                if len(input_pw) == len(password):
                    print(f"입력값: {input_pw}")
                    if input_pw == password:
                        print(">> 비밀번호 일치!")
                        play_tone(True)
                        unlock_door()
                    else:
                        print(">> 비밀번호 불일치!")
                        play_tone(False)
                        fail_count += 1
                        if fail_count >= 3:
                            print("🚨 3회 오류 경고!")
                            send_door_event("ALERT_FAIL_3")
                            # 경고음
                            for _ in range(3):
                                p.tone(7, 1, 10)
                                time.sleep(0.2)
                                p.piezo.ChangeDutyCycle(0)
                                time.sleep(0.1)
                            fail_count = 0
                    input_pw = []

                # 손 뗄 때까지 대기
                while touch.readChannel(ch) == 1:
                    time.sleep(0.05)
                time.sleep(0.1)

        time.sleep(0.05)

except KeyboardInterrupt:
    print("종료")
finally:
    try:
        pwm.stop()
        leds.off()
        p.piezo.stop() # 프로그램 끌 때만 진짜 stop
        lcd.clear()
    except: pass